In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[*]) created by <module> at C:\Users\sg\anaconda3\lib\site-packages\IPython\utils\py3compat.py:168 

In [3]:
# source pathed 
KSA_GT_path="KSA_GT.csv"
KSA_MT_path="KSA_MT.csv"
UAE_GT_MT_path="UAE_MT_GT.csv"
QTR_GT_MT_path="QTR_MT_GT.csv"

In [4]:
#loading data 
KSA_GT = spark.read.options(header='True', delimiter=',').csv(KSA_GT_path)
KSA_MT = spark.read.options(header='True', delimiter=',').csv(KSA_MT_path)
UAE_GT_MT = spark.read.options(header='True', delimiter=',').csv(UAE_GT_MT_path)
QTR_GT_MT = spark.read.options(header='True', delimiter=',').csv(QTR_GT_MT_path)
#total_df=KSA_GT.union(KSA_MT.union(UAE_GT_MT.union(QTR_GT_MT)))

In [5]:
#drop nulls rows 
KSA_GT=KSA_GT.na.drop("all")
KSA_MT=KSA_MT.na.drop("all")
UAE_GT_MT=UAE_GT_MT.na.drop("all")
QTR_GT_MT=QTR_GT_MT.na.drop("all")

In [6]:
QTR_GT_MT.show()

+----+-----+----+---------+-------+-----------+-------------------------+------------------+-----------------+------------------------+-------------+---------+----------------+---+----------+------------+-----------+------------------+-------------+--------------+---------+----------+------------+-------------+--------------------+-----------+-------+--------+-------------+----------------------+--------------------------+--------------------+----------+------------------------------+------------+-------+-------------------------------+--------------------+-----------------+
|Year|Month|Week|Frequency|Cluster|   Customer|WH Location or Store Name|Store or Warehouse|Customer SKU Code|Customer SKU Description|    Barcode #|Item Type|         Packing|UoM|QOH(Value)|QOH (Volume)|UL SKU Code|UL Master SKU Code|UL PC BarCode| UL CS BarCode|    UL BG|UL Small C|    UL Brand|  UL CPG code|         UL CPG Name|    UL Type|Pc/Case|Case/Ton|Stock in Tons|Avg monthly sales Tons|Net Avg monthly sal

In [7]:
UAE_GT_MT.show()

+----+-----+----+---------+-------+-----------+-------------------------+------------------+-----------------+------------------------+-------------+---------+----------+---+----------+------------+-----------+------------------+-------------+--------------+------------------+----------------+--------+-------------+--------------------+-----------+-------+--------+-------------+----------------------+--------------------------+--------------------+----------+------------------------------+------------+-------+-------------------------------+--------------------+-----------------+
|Year|Month|Week|Frequency|Cluster|   Customer|WH Location or Store Name|Store or Warehouse|Customer SKU Code|Customer SKU Description|    Barcode #|Item Type|   Packing|UoM|QOH(Value)|QOH (Volume)|UL SKU Code|UL Master SKU Code|UL PC BarCode| UL CS BarCode|             UL BG|      UL Small C|UL Brand|  UL CPG code|         UL CPG Name|    UL Type|Pc/Case|Case/Ton|Stock in Tons|Avg monthly sales Tons|Net Avg m

In [7]:
print(KSA_GT.count()
,KSA_MT.count()
,UAE_GT_MT.count()
,QTR_GT_MT.count())

499 499 499 7


In [ ]:
#KSA_GT trans
KSA_GT=KSA_GT.withColumnRenamed('Customer Number','Customer Code / Banner Code')\
.withColumnRenamed('Channel','Store Format')

In [ ]:
#KSA_MT trans
KSA_MT=KSA_MT.withColumnRenamed('MONTH','Month')\
.withColumnRenamed('ACCOUNT','Customer Name')\
.withColumnRenamed('ACCOUNT LOCATION','Stock Location (WH or In Store)')\
.withColumnRenamed('STORE NAME','WH Location or Store Name')\
.withColumnRenamed('Customer Account / Banner Code','Customer Code / Banner Code')

In [ ]:
#UAE_GT_MT trans
from pyspark.sql.functions import col
UAE_GT_MT=UAE_GT_MT.withColumnRenamed('Customer','Customer Name')\
.withColumnRenamed('Store or Warehouse','Stock Location (WH or In Store)')\
.withColumnRenamed('Customer Account / Banner Code','Customer Code / Banner Code')\
.withColumn('Region',col('Country'))

In [ ]:
#QTR_GT_MT trans
QTR_GT_MT=QTR_GT_MT.withColumnRenamed('Customer','Customer Name')\
.withColumnRenamed('Store or Warehouse','Stock Location (WH or In Store)')\
.withColumnRenamed('Customer Account / Banner Code','Customer Code / Banner Code')\
.withColumn('Region',col('Country'))


In [ ]:
trans_KSA_GT=KSA_GT.select('Country','Trade Format','Year','Month','Week','Customer Code / Banner Code','Customer Name','Stock Location (WH or In Store)','WH Location or Store Name','Store Format','Region')
trans_KSA_MT=KSA_MT.select('Country','Trade Format','Year','Month','Week','Customer Code / Banner Code','Customer Name','Stock Location (WH or In Store)','WH Location or Store Name','Store Format','Region')
trans_UAE_MT_GT=UAE_GT_MT.select('Country','Trade Format','Year','Month','Week','Customer Code / Banner Code','Customer Name','Stock Location (WH or In Store)','WH Location or Store Name','Store Format','Region')
trans_QTR_MT_GT=QTR_GT_MT.select('Country','Trade Format','Year','Month','Week','Customer Code / Banner Code','Customer Name','Stock Location (WH or In Store)','WH Location or Store Name','Store Format','Region')

In [ ]:
outPut=trans_KSA_GT.union(trans_KSA_MT.union(trans_UAE_MT_GT.union(trans_QTR_MT_GT)))

In [ ]:
outPut.count()

In [13]:
outPut.toPandas().to_csv('trans_output.csv', sep=';', header=True, index=False)

In [33]:
Total.toPandas().to_csv('Total.csv', sep=';', header=True, index=False)

In [39]:
KSA_GT.createOrReplaceTempView('KSA_GT_2')
KSA_MT.createOrReplaceTempView('KSA_MT_2')

In [48]:
KSA_GT_2=spark.sql("""select Country
,'Trade Format'
,Year
,Month
,Week
,'Customer Number' as CustomerCodeBannerCode
,'Customer Name'
,'Stock Location (WH or In Store)'
,'WH Location or Store Name'
,Channel as StoreFormat
,Region
 from KSA_GT_2""")

In [49]:
KSA_GT_2.toPandas()

,Country,Trade Format,Year,Month,Week,CustomerCodeBannerCode,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,StoreFormat,Region
0,KSA,Trade Format,2022,6,27,Customer Number,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,Large Supermarkets,South
1,KSA,Trade Format,2022,6,27,Customer Number,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,Large Supermarkets,South
2,KSA,Trade Format,2022,6,27,Customer Number,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,Large Supermarkets,South
3,KSA,Trade Format,2022,6,27,Customer Number,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,Large Supermarkets,South
4,KSA,Trade Format,2022,6,27,Customer Number,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,Large Supermarkets,South
...,...,...,...,...,...,...,...,...,...,...,...
494,KSA,Trade Format,2022,6,27,Customer Number,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,Small Supermarket,West
495,KSA,Trade Format,2022,6,27,Customer Number,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,Small Supermarket,West
496,KSA,Trade Format,2022,6,27,Customer Number,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,VFM,East
497,KSA,Trade Format,2022,6,27,Customer Number,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,VFM,East


In [58]:
KSA_MT_2=spark.sql("""select 
Country
,'Trade Format'
,Year
,MONTH as Month
,Week
,'Customer Account / Banner Code' as CustomerCodeBannerCode
,ACCOUNT as CustomerName
,'ACCOUNT LOCATION' as StockLocationWHorInStore
,'STORE NAME' as WHLocationorStoreName
,'Store Format'
,Region

 from KSA_MT_2""")

In [59]:
KSA_MT_2.toPandas()

,Country,Trade Format,Year,Month,Week,CustomerCodeBannerCode,CustomerName,StockLocationWHorInStore,WHLocationorStoreName,Store Format,Region
0,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
1,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
2,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
3,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
4,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
...,...,...,...,...,...,...,...,...,...,...,...
494,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
495,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
496,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
497,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN


In [60]:
Total = spark.sql("""  select Country
,'Trade Format'
,Year
,Month
,Week
,'Customer Number' as CustomerCodeBannerCode
,'Customer Name'
,'Stock Location (WH or In Store)'
,'WH Location or Store Name'
,Channel as StoreFormat
,Region
 from KSA_GT_2
 
 union all 
 select 
Country
,'Trade Format'
,Year
,MONTH as Month
,Week
,'Customer Account / Banner Code' as CustomerCodeBannerCode
,ACCOUNT as CustomerName
,'ACCOUNT LOCATION' as StockLocationWHorInStore
,'STORE NAME' as WHLocationorStoreName
,'Store Format'
,Region

 from KSA_MT_2
 """)

In [61]:
Total.toPandas()

,Country,Trade Format,Year,Month,Week,CustomerCodeBannerCode,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,StoreFormat,Region
0,KSA,Trade Format,2022,6,27,Customer Number,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,Large Supermarkets,South
1,KSA,Trade Format,2022,6,27,Customer Number,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,Large Supermarkets,South
2,KSA,Trade Format,2022,6,27,Customer Number,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,Large Supermarkets,South
3,KSA,Trade Format,2022,6,27,Customer Number,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,Large Supermarkets,South
4,KSA,Trade Format,2022,6,27,Customer Number,Customer Name,Stock Location (WH or In Store),WH Location or Store Name,Large Supermarkets,South
...,...,...,...,...,...,...,...,...,...,...,...
993,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
994,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
995,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
996,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN


In [62]:
Total.toPandas().to_csv('Total_2.csv', sep=';', header=True, index=False)

In [79]:
total=spark.sql("""select Country,
'Trade Format',
Year,
Month,
Week,
'Customer Number' as CustomerCodeBannerCode,
'Customer Name',
'Stock Location (WH or In Store)',
'WH Location or Store Name',
Channel as StoreFormat,
Region
from trans_KSA_GT
union all 
select Country
,'Trade Format'
,Year
,MONTH as Month
,Week
,'Customer Account / Banner Code'
,ACCOUNT as CustomerName
,'ACCOUNT LOCATION' as StockLocationWHorInStore
,'STORE NAME' as WHLocationorStoreName
,'Store Format'
,Region
from trans_KSA_MT
""")

In [74]:
trans_KSA_MT_DF=spark.sql("""select Country
,'Trade Format'
,Year
,MONTH as Month
,Week
,'Customer Account / Banner Code'
,ACCOUNT as CustomerName
,'ACCOUNT LOCATION' as StockLocationWHorInStore
,'STORE NAME' as WHLocationorStoreName
,'Store Format'
,Region
from trans_KSA_MT
""")

In [81]:
trans_KSA_MT_DF.toPandas()

,Country,Trade Format,Year,Month,Week,Customer Account / Banner Code,CustomerName,StockLocationWHorInStore,WHLocationorStoreName,Store Format,Region
0,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
1,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
2,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
3,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
4,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
...,...,...,...,...,...,...,...,...,...,...,...
494,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
495,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
496,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN
497,KSA,Trade Format,2022,11,45,Customer Account / Banner Code,BIN DAWOOD,ACCOUNT LOCATION,STORE NAME,Store Format,WESTERN


In [171]:
df2.count()

500